In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from faker import Faker

fake = Faker('id_ID')     
np.random.seed(42)

In [2]:
# --- Parameters ---
N_PROJECTS  = 120
N_EMPLOYEES = 150
N_SUPPLIERS = 50
N_CUSTOMERS = 80
N_PRODUCT_ORDERS = 200
N_SERVICE_ORDERS = 100
N_EQUIP     = 100
N_INTERNAL_PROJECTS = 50

# Rentang tanggal untuk data
START_DATE = pd.to_datetime("2020-01-01")
END_DATE = pd.to_datetime("2025-10-05")

def random_date(start, end, n):
    start_u = pd.Timestamp(start).value // 10**9
    end_u   = pd.Timestamp(end).value // 10**9
    return pd.to_datetime(np.random.randint(start_u, end_u, n), unit="s")

# Create External & Internal Dummy Data

In [3]:
# --- Suppliers ---
supplier_materials = ["Building Materials","Heavy Equipment","Paint & Finishing","Wood","Logistics"]
cities = ["Jakarta","Surabaya","Bandung","Medan","Makassar","Semarang","Denpasar","Balikpapan"]

suppliers = pd.DataFrame({
    "supplier_id": range(2001, 2001 + N_SUPPLIERS),
    "supplier_name": [fake.company() for _ in range(N_SUPPLIERS)],
    "material_type": np.random.choice(supplier_materials, N_SUPPLIERS),
    "contact_number": [fake.phone_number() for _ in range(N_SUPPLIERS)],
    "city": np.random.choice(cities, N_SUPPLIERS)
})

In [4]:
# --- Customers ---
customers = pd.DataFrame({
    "customer_id": range(4001, 4001 + N_CUSTOMERS),
    "company_name": [fake.company() for _ in range(N_CUSTOMERS)],
    "industry": np.random.choice(
        ["Property","Government","Retail","Energy","Infrastructure","Manufacturing"],
        N_CUSTOMERS
    ),
    "contact_person": [fake.name() for _ in range(N_CUSTOMERS)],
    "phone": [fake.phone_number() for _ in range(N_CUSTOMERS)]
})

In [5]:
# --- Employees with end date ---
roles = ["Architect", "Civil Engineer", "Project Manager","Finance","Procurement","Worker"]
employees = pd.DataFrame({
    "employee_id": range(1001, 1001 + N_EMPLOYEES),
    "name": [fake.name() for _ in range(N_EMPLOYEES)],
    "role": np.random.choice(roles, N_EMPLOYEES),
    "join_date": random_date("2020-01-01", "2024-12-31", N_EMPLOYEES),
})

# Add end_date, with 20% of employees no longer active
active_employees_count = int(N_EMPLOYEES * 0.8)
employees["end_date"] = pd.NaT # Default value for all employees
employees.loc[employees.index[active_employees_count:], "end_date"] = random_date("2025-01-01", "2025-09-30", N_EMPLOYEES - active_employees_count)


# --- Salary History ---
salary_history_data = []
for _, emp in employees.iterrows():
    # Initial salary on join_date
    initial_salary = np.random.randint(5, 35)
    salary_history_data.append({
        "employee_id": emp["employee_id"],
        "salary_effective_date": emp["join_date"],
        "salary_amount_million": initial_salary
    })
    
    # Generate random salary increases
    if pd.notna(emp["end_date"]):
        end_date = emp["end_date"]
    else:
        end_date = pd.to_datetime("2025-10-05")

    current_date = emp["join_date"] + pd.DateOffset(years=1)
    while current_date < end_date:
        if np.random.rand() > 0.6: # 40% chance of salary increase each year
            increase_amount = np.random.randint(1, 5)
            initial_salary += increase_amount
            salary_history_data.append({
                "employee_id": emp["employee_id"],
                "salary_effective_date": current_date,
                "salary_amount_million": initial_salary
            })
        current_date += pd.DateOffset(years=1)

salary_history = pd.DataFrame(salary_history_data)

# --- Monthly Attendance ---
attendance_data = []
# Generate attendance for each employee per month since joining
for _, emp in employees.iterrows():
    # Determine the end date for generating attendance data
    if pd.notna(emp["end_date"]):
        end_date = emp["end_date"]
    else:
        end_date = pd.to_datetime("2025-10-05")
        
    current_month = emp["join_date"].to_period("M").start_time
    while current_month <= end_date:
        year_month = current_month.strftime('%Y-%m')
        
        # Office worker attendance model
        if emp["role"] in ["Architect","Finance","Procurement","Project Manager"]:
            total_days_in_month = pd.Period(year_month, 'M').days_in_month
            weekend_days = sum(1 for d in pd.date_range(current_month, periods=total_days_in_month) if d.weekday() >= 5) # 5=Sat, 6=Sun
            working_days = total_days_in_month - weekend_days
            absent_days = np.random.randint(0, 3)
            sick_leave_days = np.random.randint(0, 2)
            present_days = working_days - absent_days - sick_leave_days
            
        # Field worker attendance model
        else: # Worker
            total_days_in_month = pd.Period(year_month, 'M').days_in_month
            working_days = total_days_in_month
            absent_days = np.random.randint(0, 8)
            sick_leave_days = np.random.randint(0, 4)
            present_days = working_days - absent_days - sick_leave_days
        
        attendance_data.append({
            "employee_id": emp["employee_id"],
            "attendance_date": current_month,
            "present_days": max(0, present_days),
            "absent_days": max(0, absent_days),
            "sick_leave_days": max(0, sick_leave_days)
        })
        current_month += pd.DateOffset(months=1)
        
monthly_attendance = pd.DataFrame(attendance_data)
monthly_attendance["attendance_date"] = monthly_attendance["attendance_date"].dt.to_period("M")

In [6]:
# --- Projects ---
projects = pd.DataFrame({
    "project_id": range(1, N_PROJECTS + 1),
    "project_name": [f"Project {fake.city()}" for _ in range(N_PROJECTS)],
    "customer_id": np.random.choice(customers["customer_id"], N_PROJECTS),
    "start_date": random_date("2024-01-01", "2025-01-01", N_PROJECTS),
    "end_date": random_date("2025-02-01", "2026-12-31", N_PROJECTS),
    "budget_million": np.random.randint(5, 50, N_PROJECTS),
    "status": np.random.choice(["Planning","Ongoing","Completed"], N_PROJECTS)
})

# Add employee assignment to projects
project_employees = []
for project_id in projects["project_id"].unique():
    # Assign employees to each project
    num_employees = np.random.randint(2, 6)
    assigned_employees = np.random.choice(employees["employee_id"], num_employees, replace=False)
    for emp_id in assigned_employees:
        project_employees.append({"project_id": project_id, "employee_id": emp_id})

project_assignments = pd.DataFrame(project_employees)

In [7]:
# --- Product Orders ---
items = ["Steel Bar","Cement","Bricks","Paint","Wood","Nails","Pipes","Cables"]

product_orders = pd.DataFrame({
    "order_id": range(5001, 5001 + N_PRODUCT_ORDERS),
    "customer_id": np.random.choice(customers["customer_id"], N_PRODUCT_ORDERS),
    "supplier_id": np.random.choice(suppliers["supplier_id"], N_PRODUCT_ORDERS),
    "order_date": random_date("2024-02-01", "2025-09-01", N_PRODUCT_ORDERS),
    "item": np.random.choice(items, N_PRODUCT_ORDERS),
    "quantity": np.random.randint(50, 5000, N_PRODUCT_ORDERS),
    "unit_price": np.random.randint(3000, 60000, N_PRODUCT_ORDERS),
    "status": np.random.choice(["Ordered","Shipped","Delivered","Cancelled"], N_PRODUCT_ORDERS)
})

In [8]:
# --- Equipment with supplier_id ---
equipment = pd.DataFrame({
    "equipment_id": range(3001, 3001 + N_EQUIP),
    "name": np.random.choice(["Excavator","Bulldozer","Crane","Dump Truck","Concrete Mixer","Loader","Forklift"], N_EQUIP),
    "purchase_date": random_date("2018-01-01", "2024-12-31", N_EQUIP),
    "last_maintenance": random_date("2024-01-01", "2025-09-01", N_EQUIP),
    "condition": np.random.choice(["Good","Needs Service","Critical"], N_EQUIP)
})

# Add equipment supplier relationship
equipment["supplier_id"] = np.random.choice(suppliers["supplier_id"], N_EQUIP)

In [9]:
# --- Project Equipment ---
project_equipment_data = []
for project_id in projects["project_id"].unique():
    num_equipment = np.random.randint(1, 5)
    assigned_equipment = np.random.choice(equipment["equipment_id"], num_equipment, replace=False)
    for eq_id in assigned_equipment:
        project_equipment_data.append({
            "project_id": project_id,
            "equipment_id": eq_id,
            "assignment_date": random_date("2024-01-01", "2025-09-01", 1)[0]
        })
project_equipment = pd.DataFrame(project_equipment_data)

In [10]:
# --- Internal Projects ---
internal_projects = pd.DataFrame({
    "internal_project_id": range(7001, 7001 + N_INTERNAL_PROJECTS),
    "project_name": [f"Internal Project {i}" for i in range(1, N_INTERNAL_PROJECTS+1)],
    "employee_id": np.random.choice(employees["employee_id"], N_INTERNAL_PROJECTS),
    "equipment_id": np.random.choice(equipment["equipment_id"], N_INTERNAL_PROJECTS),
    "start_date": random_date("2024-01-01", "2025-01-01", N_INTERNAL_PROJECTS),
    "end_date": random_date("2025-02-01", "2025-12-31", N_INTERNAL_PROJECTS),
    "description": np.random.choice(["Internal Training","Equipment Testing","Maintenance Project","R&D Work"], N_INTERNAL_PROJECTS)
})

# Show Output

In [11]:
# --- Display the first 5 rows of each DataFrame for verification ---
print("Employees:")
employees.head()

Employees:


,employee_id,name,role,join_date,end_date
0,1001,Irma Marpaung,Worker,2024-03-29 16:56:43,NaT
1,1002,Humaira Mandala,Project Manager,2021-03-14 03:36:26,NaT
2,1003,Keisha Utami,Civil Engineer,2021-07-17 08:20:37,NaT
3,1004,"R. Zulaikha Usada, S.Psi",Finance,2022-01-25 17:08:06,NaT
4,1005,Bakda Riyanti,Architect,2020-01-23 22:58:29,NaT


In [12]:
print("\nSalary History:")
salary_history.head()


Salary History:


,employee_id,salary_effective_date,salary_amount_million
0,1001,2024-03-29 16:56:43,16
1,1002,2021-03-14 03:36:26,16
2,1002,2025-03-14 03:36:26,17
3,1003,2021-07-17 08:20:37,30
4,1003,2023-07-17 08:20:37,32


In [13]:
print("\nMonthly Attendance:")
monthly_attendance.head()


Monthly Attendance:


,employee_id,attendance_date,present_days,absent_days,sick_leave_days
0,1001,2024-03,28,0,3
1,1001,2024-04,29,1,0
2,1001,2024-05,29,0,2
3,1001,2024-06,29,0,1
4,1001,2024-07,24,5,2


In [14]:
print("\nProjects:")
projects.head()


Projects:


,project_id,project_name,customer_id,start_date,end_date,budget_million,status
0,1,Project Lhokseumawe,4019,2024-10-22 19:36:40,2026-09-12 03:29:11,48,Completed
1,2,Project Cimahi,4056,2024-12-21 21:05:09,2026-02-22 06:21:09,27,Completed
2,3,Project Lhokseumawe,4014,2024-07-17 07:20:56,2025-02-25 21:50:59,37,Planning
3,4,Project Tidore Kepulauan,4033,2024-01-10 17:46:58,2025-12-19 14:03:54,20,Ongoing
4,5,Project Cirebon,4010,2024-04-18 05:19:11,2025-03-12 05:55:19,28,Completed


In [15]:
print("\nProject Assignments:")
project_assignments.head()


Project Assignments:


,project_id,employee_id
0,1,1114
1,1,1078
2,1,1060
3,1,1059
4,1,1016


In [16]:
print("\nProject Equipment:")
project_equipment.head()


Project Equipment:


,project_id,equipment_id,assignment_date
0,1,3050,2024-07-24 08:52:06
1,1,3039,2025-01-11 00:21:55
2,1,3070,2024-03-01 06:24:50
3,2,3053,2025-01-27 06:40:03
4,2,3069,2024-09-09 04:30:15


In [17]:
print("\nEquipment:")
equipment.head()


Equipment:


,equipment_id,name,purchase_date,last_maintenance,condition,supplier_id
0,3001,Dump Truck,2023-12-17 04:16:54,2025-01-10 13:46:55,Critical,2050
1,3002,Excavator,2020-06-03 04:15:18,2024-12-12 10:17:07,Critical,2006
2,3003,Dump Truck,2021-05-12 11:26:34,2024-05-12 18:37:12,Good,2022
3,3004,Dump Truck,2020-04-26 10:03:53,2024-09-08 13:49:49,Good,2040
4,3005,Concrete Mixer,2022-04-13 07:59:12,2024-06-22 08:47:59,Good,2009


In [18]:
print("\nSuppliers:")
suppliers.head()


Suppliers:


,supplier_id,supplier_name,material_type,contact_number,city
0,2001,Perum Natsir Halim,Wood,+62 (713) 560-7990,Medan
1,2002,PD Setiawan Tbk,Logistics,084 162 1472,Denpasar
2,2003,CV Kuswandari,Paint & Finishing,(063) 107 7487,Medan
3,2004,CV Sihombing,Logistics,+62-0180-435-1993,Denpasar
4,2005,CV Rahayu (Persero) Tbk,Logistics,+62 (99) 758-3073,Medan


In [19]:
print("\nCustomers:")
customers.head()


Customers:


,customer_id,company_name,industry,contact_person,phone
0,4001,Perum Rahimah Tbk,Property,Syahrini Winarsih,+62 (037) 362 5569
1,4002,UD Fujiati Saefullah,Energy,"Anastasia Zulkarnain, S.Sos",+62 (0536) 986-5176
2,4003,PD Nashiruddin (Persero) Tbk,Energy,Narji Haryanto,(005) 839-5645
3,4004,PD Hasanah Yolanda (Persero) Tbk,Energy,Ami Lazuardi,(096) 904 1345
4,4005,Perum Winarsih Widodo,Infrastructure,Anom Hartati,+62 (0875) 697-8498


In [20]:
print("\nProduct Orders:")
product_orders.head()


Product Orders:


,order_id,customer_id,supplier_id,order_date,item,quantity,unit_price,status
0,5001,4039,2021,2025-05-27 05:49:59,Paint,102,16487,Ordered
1,5002,4005,2020,2024-12-27 22:37:04,Cement,2326,43139,Shipped
2,5003,4057,2036,2025-07-14 06:59:18,Cables,2737,56137,Cancelled
3,5004,4015,2026,2024-05-05 09:25:31,Paint,2894,37037,Shipped
4,5005,4016,2047,2025-06-18 09:25:08,Wood,4936,15898,Shipped


In [21]:
print("\nInternal Projects:")
internal_projects.head()


Internal Projects:


,internal_project_id,project_name,employee_id,equipment_id,start_date,end_date,description
0,7001,Internal Project 1,1029,3077,2024-06-20 07:08:55,2025-03-24 18:41:15,Maintenance Project
1,7002,Internal Project 2,1043,3035,2024-10-24 08:42:04,2025-11-02 23:33:45,Maintenance Project
2,7003,Internal Project 3,1065,3017,2024-01-01 16:24:09,2025-02-20 16:03:15,Maintenance Project
3,7004,Internal Project 4,1131,3030,2024-01-09 15:28:38,2025-06-30 08:35:17,R&D Work
4,7005,Internal Project 5,1096,3018,2024-07-05 02:46:10,2025-09-08 02:56:56,Equipment Testing


# Save All Dummy Data to CSV

In [22]:
import os

def save_all_to_csv(data_dict, folder_path="data_dummy"):
    """
    Simpan seluruh DataFrame dalam dictionary ke file CSV.
    
    Parameters
    ----------
    data_dict : dict
        Dictionary dengan format {"nama_file": dataframe}
    folder_path : str, optional
        Folder tujuan penyimpanan (default: 'data_dummy')
    """
    # buat folder jika belum ada
    os.makedirs(folder_path, exist_ok=True)
    
    for name, df in data_dict.items():
        file_path = os.path.join(folder_path, f"{name}.csv")
        df.to_csv(file_path, index=False)
        print(f"Saved: {file_path}")

In [27]:
# Kumpulkan semua DataFrame
all_tables = {
    "employees": employees,
    "salary_history": salary_history,
    "monthly_attendance": monthly_attendance,
    "projects": projects,
    "project_assignments": project_assignments,
    "project_equipment": project_equipment, 
    "equipment": equipment,
    "suppliers": suppliers,
    "customers": customers,
    "product_orders": product_orders,
    "internal_projects": internal_projects
}

# Simpan ke folder 'data_dummy'
save_all_to_csv(all_tables, folder_path="new")

Saved: new\employees.csv
Saved: new\salary_history.csv
Saved: new\monthly_attendance.csv
Saved: new\projects.csv
Saved: new\project_assignments.csv
Saved: new\project_equipment.csv
Saved: new\equipment.csv
Saved: new\suppliers.csv
Saved: new\customers.csv
Saved: new\product_orders.csv
Saved: new\internal_projects.csv
